In [ ]:
import sys
sys.path.append('/opt/miniconda3/lib/python3.7/site-packages')
from urllib.request import urlopen
import urllib
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
import numpy as np
from selenium import webdriver
from time import sleep

In [ ]:
colName = ['品牌', '行业','LOGO', '链接']
df = pd.DataFrame(columns=colName)

# 爬取品牌及行业信息
driver = webdriver.Chrome()
driver.get("https://www.digitaling.com/dindex/brand")

driver.find_element_by_class_name('travel').click()
driver.implicitly_wait(10) # 隐形等待
# 添加品牌信息
names = driver.find_elements_by_xpath("//span[@class='name']")
logos = driver.find_elements_by_xpath("//div[@class='daoy_company']/a/img")
product = "航旅"
links = driver.find_elements_by_xpath("//div[@class='daoy_company']/a")

for i in range(0,100):
    name = names[i].text
    logo = logos[i].get_attribute("src")
    link = links[i].get_attribute("href")
    print(name,logo,link)
    df.loc[i] = [name,product,logo,link]
df.to_csv('travel.csv',encoding='utf-8')

driver.close()

In [ ]:
# 数据整合
car = pd.read_csv('data/merge/car.csv')
cloth = pd.read_csv('data/merge/cloth.csv')
cosmetic = pd.read_csv('data/merge/cosmetic.csv')
digital = pd.read_csv('data/merge/digital.csv')
entertain = pd.read_csv('data/merge/entertain.csv')
financial = pd.read_csv('data/merge/financial.csv')
food = pd.read_csv('data/merge/food.csv')
luxury = pd.read_csv('data/merge/luxury.csv')
sall = pd.read_csv('data/merge/sall.csv')
sport = pd.read_csv('data/merge/sport.csv')
travel = pd.read_csv('data/merge/travel.csv')
web = pd.read_csv('data/merge/web.csv')
wine = pd.read_csv('data/merge/wine.csv')

total = car.append(cloth).append(cosmetic).append(digital).append(entertain).append(financial).append(food).append(luxury).append(sall).append(sport).append(travel).append(web).append(wine)

total.to_csv('data/totalAnalyse.csv',encoding='utf-8')

In [ ]:
def getIntroduction(url):
    driver = webdriver.Chrome()
    driver.get(url)

    # 品牌主页，简介内容
    try:
        driver.find_element_by_link_text('展开').click()
    except BaseException:
        pass
        text = "无简介内容"
    else:
        text = driver.find_element_by_class_name('real_intro_txt').text
        text = text.strip(' 收起') # 删除多余的“收起”
    
    driver.close()

    return text

In [ ]:
# data = pd.read_csv('data/total.csv')
# data['简介']=''

for i in range(620,len(data)):
    link = data.at[i,'链接']
    text = getIntroduction(link)
    data.loc[i,'简介'] = text
    print(i)
    print(text)

data.to_csv('data/data_introduction.csv',encoding='utf-8')

In [ ]:
def getAttention(tab):
    global flag
    driver = webdriver.Chrome()
    driver.get("https://www.digitaling.com/dindex/brand")

    driver.find_element_by_class_name(tab).click()
    driver.implicitly_wait(10) # 隐形等待
    # 关注度
    names = driver.find_elements_by_class_name('attention')
    for i in range(0,len(names)):
        name = names[i].text
        
        print(flag,name)
        data.loc[flag,'关注度'] = name
        flag = flag + 1

    driver.close()

In [ ]:
data['关注度'] = ''
tabList = ['car','cloth','cosmetic','digital','entertain','financial','food','luxury','sall','sport','travel','web','wine']

flag = 0
for tab in tabList:
    getAttention(tab)
data.to_csv('data/data_attention.csv',encoding='utf-8')


In [ ]:
def findNum(name):
    colName = ['项目数', '首页收录','获奖数', '项目品质','关注度']
    df = pd.DataFrame(columns=colName)

    # 爬取品牌及行业信息
    driver = webdriver.Chrome()
    driver.get("https://www.digitaling.com/dindex/brand")

    driver.find_element_by_class_name(name).click()
    driver.implicitly_wait(10) # 隐形等待
    # 添加品牌信息
    projects = driver.find_elements_by_xpath("//span[@class='project']")
    collects = driver.find_elements_by_xpath("//span[@class='collect']")
    numbers = driver.find_elements_by_xpath("//span[@class='collect']/following-sibling::span")
    qualitys = driver.find_elements_by_xpath("//span[@class='quality']")
    attentions = driver.find_elements_by_xpath("//span[@class='attention']")

    for i in range(0,100):
        try:
            project = projects[i].text
            collect = collects[i].text
            number = numbers[i].text
            quality = qualitys[i].text
            attention = attentions[i].text
       # print(project,collect,number,quality,attention)
            df.loc[i] = [project,collect,number,quality,attention]
        except BaseException:
            pass
        else:
            pass
    df.to_csv(name+'2.csv',encoding='utf-8')

    driver.close()

In [ ]:
def mergeData(name):
    df = pd.read_csv('data/industry/'+name+'.csv')  
    dd = pd.read_csv(name+'2.csv')
    data = pd.merge(df, dd)  # pandas csv表左连接
    data = data.drop(columns=['LOGO', '链接'])
    data.to_csv('123/'+name+'.csv',encoding='utf-8')


In [ ]:
brandName = ['car','cloth','cosmetic','digital','entertain','financial','food','luxury','sall','sport','travel','web','wine']

for name in brandName:
    mergeData(name)

In [ ]:
data = pd.read_csv('data/total_intro.csv')  
data = data.drop(columns=['ID', 'LOGO','链接'])
data.to_csv('data/totalIntroduction.csv',encoding='utf-8')

In [ ]:
def getProjectNum(name):
    # 判断是否存在该文件夹
    path = "/Volumes/Backup Plus/brandData/"+name 
    num = 0
    filenum = 0
    isDir = False
    isDir = os.path.exists(path)
    # 存在 获得其中文件的数量
    if isDir:
        for lists in os.listdir(path):
            sub_path = os.path.join(path, lists)
            if os.path.isfile(sub_path):
                filenum = filenum+1
        
    print('filenum: ',filenum)
    # 不存在 记为零
    return filenum


In [ ]:
data = pd.read_csv('../data/totalAnalyse.csv')
for i in range(0,len(data)):
    name = data.loc[i,"品牌"]
    fileNum = getProjectNum(name)
    data.loc[i,'项目图片数量'] = fileNum

data.to_csv('../data/totalAnalyse.csv',encoding='utf-8',index=False)